### Testing Playground for P0 Programs

In [1]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString
from ST import printSymTab

def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            write: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            read: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))

Importing Jupyter notebook from P0.ipynb
Importing Jupyter notebook from SC.ipynb
Importing Jupyter notebook from ST.ipynb



Write, compile and execute P0 programs for testing using the following steps:
1. Define a string constant containing a P0 program to be validated in the cell below.
2. Update the `compileString` parameter to accept the string constant in the cell below.
3. Select `Kernel > Restart & Run All` to view and execute the target WebAssembly program.

In [29]:
# Concatenation array with single element array literal
test1 = """
program p
    var a: [1..3] → integer
    var b: [1..3] → integer
    a := [3, 7, 11]
    b := [4, 8, 12]
    a := a + [1]
    write(a[4])
"""
compileString(test1, "test1.wat")
!wat2wasm --enable-bulk-memory test1.wat || rm test1.wasm
runwasm("test1.wasm")

<IPython.core.display.Javascript object>

In [31]:
# Concatenating array with multiple element array literal
test2 = """
program p
    var a: [1..3] → integer
    var b: [1..3] → integer
    a := [3, 7, 11]
    b := [4, 8, 12]
    a := a + [1, 5, 9]
    write(a[4])
    write(a[5])
    write(a[6])
"""
compileString(test2, "test2.wat")
!wat2wasm --enable-bulk-memory test2.wat || rm test2.wasm
runwasm("test2.wasm")

<IPython.core.display.Javascript object>

In [34]:
#Update this line to see WebAssembly of a specific test case
!cat -n "test1.wat"

     1	(module
     2	(import "P0lib" "write" (func $write (param i32)))
     3	(import "P0lib" "writeln" (func $writeln))
     4	(import "P0lib" "read" (func $read (result i32)))
     5	(global $_memsize (mut i32) i32.const 0)
     6	(func $program
     7	(local $a i32)
     8	(local $b i32)
     9	(local $0 i32)
    10	(local $_fp i32)
    11	global.get $_memsize
    12	local.set $_fp
    13	global.get $_memsize
    14	i32.const 12
    15	i32.add
    16	local.tee $a
    17	global.set $_memsize
    18	global.get $_memsize
    19	i32.const 12
    20	i32.add
    21	local.tee $b
    22	global.set $_memsize
    23	i32.const 1
    24	i32.const 1
    25	i32.sub
    26	i32.const 4
    27	i32.mul
    28	local.get $a
    29	i32.add
    30	i32.const 3
    31	i32.store
    32	i32.const 2
    33	i32.const 1
    34	i32.sub
    35	i32.const 4
    36	i32.mul
    37	local.get $a
    38	i32.add
    39	i32.const 7
    40	i32.store
    41	i32.const 3
    42	i32.const 1
    43	i32.sub
    44	i32.const 4
